In [1]:
import sys
import numpy as np
import pandas as pd
from tqdm import tqdm
import copy

import csv

import itertools
import collections
import time
import random

In [2]:
pat_table_imported = pd.read_csv('../saved_data/patients.csv')
# pat_table_imported

In [3]:
adm_table_imported = pd.read_csv('../saved_data/admissions.csv')
# adm_table_imported

In [4]:
icu_table_imported = pd.read_csv('../saved_data/icustays.csv')
# icu_table_imported

In [5]:
pat_fields = ['uniquepid','patientunitstayid','patienthealthsystemstayid','hospitaladmitoffset','unitvisitnumber','unitdischargestatus']

In [ ]:
adm_fields = ['patientunitstayid','admitdxenteredoffset','admitdxpath']

Code for 'patient' table starts here

In [6]:
pd_merged = pd.merge(adm_table_imported, pat_table_imported, on='subject_id', how='outer')
pat_table_og = pd.merge(icu_table_imported, pd_merged, on='hadm_id', how='outer')
pat_table_og = pat_table_og.sort_values(by=['hadm_id'])
# pat_table_og

In [7]:
pat_table_og['visits'] = pat_table_og.groupby("subject_id_x")["subject_id_x"].transform('count')
# pat_table_og.columns

In [8]:
pat_table_og['intime'] = pd.to_datetime(pat_table_og['intime'])
pat_table_og['admittime'] = pd.to_datetime(pat_table_og['admittime'])


In [9]:
pat_table = pd.DataFrame(columns=pat_fields)
for i in range(len(pat_table_og)):
    curr_row = len(pat_table.index)
    pat_table.loc[curr_row, "uniquepid"] = pat_table_og.loc[i, "subject_id_x"]
    pat_table.loc[curr_row, "patientunitstayid"] = pat_table_og.loc[i, "icustay_id"]
    pat_table.loc[curr_row, "patienthealthsystemstayid"] = pat_table_og.loc[i, "hadm_id"]
    pat_table.loc[curr_row, "unitvisitnumber"] = pat_table_og.loc[i, "visits"]
pat_table['hospitaladmitoffset'] = (pat_table_og['admittime'] - pat_table_og['intime']).dt.total_seconds()/60
pat_table['unitdischargestatus'] = np.where(pat_table_og['hospital_expire_flag']==1, 'Expired', np.where(pat_table_og['hospital_expire_flag']==0, 'Alive', None))

# pat_table

In [10]:
pat_table['uniquepid'] = pat_table['uniquepid'].astype("string")
pat_table['patientunitstayid'] = pat_table['patientunitstayid'].astype("int")
pat_table['patienthealthsystemstayid'] = pat_table['patienthealthsystemstayid'].astype("int")
pat_table['unitvisitnumber'] = pat_table['unitvisitnumber'].astype("int")
pat_table['unitdischargestatus'] = pat_table['unitdischargestatus'].astype("string")

In [11]:
pat_table['hospitaladmitoffset'] = round(pat_table['hospitaladmitoffset'], 0)
pat_table['hospitaladmitoffset'] = pat_table['hospitaladmitoffset'].astype("int")

In [12]:
pat_table.to_csv('../saved_data/patient.csv', index=False)

Code for 'admissiondx' table starts here

In [203]:
# pnt = pd.read_csv("../saved_data/patient.csv")
# pnt